In [ ]:
import cv2
import time
import numpy as np
import pyrealsense2 as rs

# Initialize the RealSense camera and start streaming
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
pipeline.start(config)

# Create a font for the timestamp text
font = cv2.FONT_HERSHEY_SIMPLEX

# Main loop to process camera frames
while True:
    # Wait for a new camera frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    # Convert the color frame to a NumPy array for processing
    color_image = np.asanyarray(color_frame.get_data())

    # Get the current timestamp and format it as a string
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

    # Draw the timestamp text onto the color image
    cv2.putText(color_image, timestamp, (10, 30), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Show the resulting image with the timestamp overlay
    cv2.imshow("RealSense Camera", color_image)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
cv2.destroyAllWindows()

In [ ]:
import pyrealsense2 as rs

# Initialize the RealSense pipeline and configure it to enable the gyro stream
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.gyro, rs.format.motion_xyz32f, 200)

# Start the pipeline
pipeline.start(config)

# Main loop to process gyro frames
while True:
    # Wait for a new gyro frame
    frames = pipeline.wait_for_frames()
    print("hi")
    gyro_frame = frames.first_or_default(rs.stream.gyro)

    # Extract the gyro values from the gyro frame
    if gyro_frame:
        gyro_data = gyro_frame.as_motion_frame().get_motion_data()
        gyro_x, gyro_y, gyro_z = gyro_data.x, gyro_data.y, gyro_data.z
        print(f"Gyro X: {gyro_x}, Gyro Y: {gyro_y}, Gyro Z: {gyro_z}")

    # Break the loop if the 'q' key is pressed
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

# Stop the pipeline and release resources
pipeline.stop()

In [ ]:
import pyrealsense2 as rs

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.gyro)

pipeline.start(config)

try:
    while True:
        frames = pipeline.wait_for_frames()

        # Get the gyro frame
        gyro_frame = frames.first_or_default(rs.stream.gyro)

        # Check if the frame is valid
        if gyro_frame:
            # Get the gyro data
            gyro_data = gyro_frame.as_motion_frame().get_motion_data()

            # Print the gyro data
            print(f"Gyro X: {gyro_data.x}")
            print(f"Gyro Y: {gyro_data.y}")
            print(f"Gyro Z: {gyro_data.z}")
finally:
    pipeline.stop()

In [ ]:
import cv2
import time
import numpy as np
import pyrealsense2 as rs

# Initialize the RealSense camera and start streaming
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.gyro, rs.format.motion_xyz32f, 200)


pipeline.start(config)

# Create a font for the timestamp text
font = cv2.FONT_HERSHEY_SIMPLEX

# Main loop to process camera frames
while True:

    # Wait for a new camera frame
    frames = pipeline.wait_for_frames(timeout_ms=10000)
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    gyro_frame = frames.first_or_default(rs.stream.gyro)

    # Convert the color frame to a NumPy array for processing
    color_image = np.asanyarray(color_frame.get_data())

    # Get the current timestamp and format it as a string
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

    gyro_data = gyro_frame.as_motion_frame().get_motion_data()
    gyro_x, gyro_y, gyro_z = gyro_data.x, gyro_data.y, gyro_data.z

    # Draw the timestamp text onto the color image
    cv2.putText(color_image, timestamp, (10, 30), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(color_image, gyro_x, (10, 30), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(color_image, gyro_y, (10, 30), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(color_image, gyro_z, (10, 30), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Show the resulting image with the timestamp overlay
    cv2.imshow("RealSense Camera", color_image)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
cv2.destroyAllWindows()

In [ ]:
import pyrealsense2 as rs
import math


def initialize_camera():
    # start the frames pipe
    p = rs.pipeline()
    conf = rs.config()
    conf.enable_stream(rs.stream.accel)
    conf.enable_stream(rs.stream.gyro)
    prof = p.start(conf)
    return p


p = initialize_camera()

first = True
alpha = 0.98

try:
    while True:
        f = p.wait_for_frames()

        # gather IMU data
        accel = f[0].as_motion_frame().get_motion_data()
        gyro = f[1].as_motion_frame().get_motion_data()

        ts = f.get_timestamp()

        # calculation for the first frame
        if first:
            first = False
            last_ts_gyro = ts

            # accelerometer calculation
            accel_angle_z = math.degrees(math.atan2(accel.y, accel.z))
            accel_angle_x = math.degrees(
                math.atan2(accel.x, math.sqrt(accel.y * accel.y + accel.z * accel.z))
            )
            accel_angle_y = math.degrees(math.pi)

            continue

        # calculation for the second frame onwards

        # gyrometer calculations
        dt_gyro = (ts - last_ts_gyro) / 1000
        last_ts_gyro = ts

        gyro_angle_x = gyro.x * dt_gyro
        gyro_angle_y = gyro.y * dt_gyro
        gyro_angle_z = gyro.z * dt_gyro

        dangleX = gyro_angle_x * 57.2958
        dangleY = gyro_angle_y * 57.2958
        dangleZ = gyro_angle_z * 57.2958

        totalgyroangleX = accel_angle_x + dangleX
        totalgyroangleY = accel_angle_y + dangleY
        totalgyroangleZ = accel_angle_z + dangleZ

        # accelerometer calculation
        accel_angle_z = math.degrees(math.atan2(accel.y, accel.z))
        accel_angle_x = math.degrees(
            math.atan2(accel.x, math.sqrt(accel.y * accel.y + accel.z * accel.z))
        )
        accel_angle_y = math.degrees(math.pi)

        # combining gyrometer and accelerometer angles
        combinedangleX = totalgyroangleX * alpha + accel_angle_x * (1 - alpha)
        combinedangleZ = totalgyroangleZ * alpha + accel_angle_z * (1 - alpha)
        combinedangleY = totalgyroangleY

        print(
            "Angle -  X: "
            + str(round(combinedangleX, 2))
            + "   Y: "
            + str(round(combinedangleY, 2))
            + "   Z: "
            + str(round(combinedangleZ, 2))
        )

finally:
    p.stop()